<div style="text-align: center;"><h1> 🔊 Audio MNIST Tutorial - PyTorch 🔊 </h1></div>

<div style="text-align: center; margin-top: 50px"> <img src="./images/5Tz.gif" width="300"> </div>

        WELCOME !

Bonjour et bienvenue dans ce tuto qui va t'apprendre à utiliser tout un tas d'outils utiles pour créer une IA capable de déterminer le chiffre qu'une personne dira.

Avant toute chose, pour suivre ce tuto, il te faut un minimum de connaissance dans le traitement audio, le fonctionnement des réseaux de neurones (plus particulièrement les CNNs) et Python, évidemment. Donc si tu n'as pas fait les tutos concernant ces domaines avant celui-ci, je te conseille vivement d'aller les faire, car je n'ai pas l'intention de ré expliquer ce qui a déjà été expliqué.

Si tu lis cette ligne, je suppose que tu as les connaissances nécessaires pour commencer ce tuto ... Ou alors que tu t'en fiche ... Bah, peu importe, dans tous les cas, c'est parti !

<div style="text-align: center; margin-top: 50px"> <img src="./images/32R.gif" width="400"> </div>

        LE CONTENU

Pour bien démarrer ce tutoriel, je vais tout d'abord t'expliquer comment celui-ci va se dérouler.

En premier lieu, sache que ce tuto est découpé en deux parties. La première, celle où tu te trouves, va t'aider à comprendre que la création d'une IA ne se limite pas qu'à ... la création de l'IA. Attends, je vais t'expliquer.

Comme tu dois sûrement le savoir, pour créer une IA en utilisant la technologie du deep learning, il faut trois grandes étapes : créer le modèle d'apprentissage de l'IA, la faire apprendre et la tester. Jusque-là, normalement, je ne t'ai pas perdu. Et, où je veux en venir, c'est que parmi ces trois grandes étapes, les deux dernières supposent quelques choses d'importants. C'est qu'ils nous faut des données. Beaucoup, beaucoup de données. Et toutes ces données, ils faut les créer et les traiter pour pouvoir les utiliser. Et ce travail sur ces données, nous allons le faire dans la première partie de ce tuto.

La seconde partie, comme tu dois t'en douter, va te permettre de comprendre le cœur du sujet. La création, l'apprentissage et le test d'un modèle.

Il existe plusieurs technologies sur Python qui permettent de faire du deep learning (créer des modèles, les entraîner, ect...). Mais ce tuto sera consacré à l'utilisation de PyTorch. PyTorch est une bibliothèque open source de machine learning qui s'appuie sur Torch développée par Facebook.

PyTorch permet d'effectuer les calculs tensoriels nécessaires notamment pour le deep learning. Ces calculs sont optimisés et effectués soit par le processeur (CPU) soit, lorsque c'est possible, par un processeur graphique (GPU) supportant CUDA.

PyTorch permet de :

- manipuler des tenseurs, de les échanger facilement avec Numpy et d'effectuer des calculs efficaces sur CPU ou GPU (par exemple, des produits de matrices ou des convolutions);
- calculer des gradients pour appliquer facilement des algorithmes d'optimisation par descente de gradient. PyTorch utilise la bibliothèque autograd.


        LES DONNÉES

Maintenant que tu sais ce qui t'attend, on va se lancer dans le traitement des données. Comme on n'est pas des monstres, on a quand même préparé le terrain en fournissant des données brutes pour ce tutoriel. 

Donc la création des données, c'est fait. Mais comme je viens de le dire, ce sont des données brutes, donc on va devoir les traités pour les rendre utilisables dans la deuxième partie du tuto. Ces données, tu peux les trouver dans le dossier AudioMNIST_data. Ce sont des enregistrements audio de 60 personnes différentes, rangés dans 60 dossiers différents numérotées de 01 à 60. Chaque personne a permis de créer 500 enregistrements différents avec, à chaque fois, 50 audios pour chaque chiffre de 0 à 9 en anglais. Le nom des audios permet de retrouver leurs origines, par exmple l'audio 4_01_14.wav correspond au 14ème enregistrements de la personne 01 qui prononce le chiffre 4. Au total, cela nous fait 30 000 enregistrements pour notre application. 

On peut aussi trouver dans ce dossier, le fichier audioMNIST_meta.txt qui nous donne des informations sur chaque personne et leurs conditions d'enregistrements. Vu que nous voulons classer les enregistrements en fonction du chiffre qui est prononcé, ce fichier ne nous sera pas utile.

<div style="text-align: center; margin-top: 50px"> <img src="./images/gee-data-entry.gif" width="300"> </div>

        LES BIBLIOTHÈQUES

Il existe énormément de bibliothèques en Python capable de faire du traitement de données comme Pandas, NumPy, Matplotlib, SciPy, SciKit-Learn, ect... Et pour ce tuto, nous allons utiliser les deux plus simples à prendre en main, NumPy et Matplotlib.

- Numpy est une bibliothèque qui est destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.

- Matplotlib, quant à elle, est plutôt destinée à tracer et visualiser des données sous forme de graphiques. Elle peut être combinée avec les bibliothèques NumPy et SciPy. Elle nous sera très utile pour visualiser et comprendre les données qu'on va traiter.

De plus, nous allons utiliser librosa que vous avez déjà utiliser lors des tutos précédents et qui va nous servir à transformer les audios en données plus utiles pour notre modèle.

Nous allons aussi utiliser d'autres bibliothèques comme cv2 qui va être utilisé pour du traitement d'images, et random qui va nous permettre de générer des nombres pseudos-randoms.

Et enfin, nous allons aussi utiliser le module OS qui fournit des fonctions d’interaction avec le système d’exploitation. Ce module nous sera utile pour récupérer les données brutes dans le programme.


In [ ]:
import os
import numpy as np
from numpy import savetxt

import random
import cv2
from IPython.display import clear_output

import librosa
import librosa.display

import matplotlib.pyplot as plt

        RÉCUPÉRATION DES DONNÉES
        
Tout d'abord, nous allons récupérer les chemins relatifs de tous les enregistrements audios. Nous allons les stocker dans un dictionnaire afin de leur associé, en même temps que leur stockage, une valeur "train" ou "test" qui va définir si le fichier associé à ce chemin sera utilisé plus tard pour entraîner le modèle ou tester le modèle. En recherchant sur le net les différents projets de deep learning disponibles, on peut constater que pour un jeu de donnée définit, 80% d'entre elle sont utilisées pour l'entraînement, et 20% pour le test. Et c'est exactement ce que nous allons faire ici.

In [ ]:
# Nom du fichier dans lequel les données brutes sont stockées
data_dir = './AudioMNIST_data/'

# Tableau qui va contenir tout les chemins vers toutes les données brutes
file_names = {}

test_indices = random.sample(range(0, 30000), int((30000*20)/100))
test_indices.sort()

i = j = 0

for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        file_label = filename[0]
        # Si le fichier est audioMNIST_meta.txt alors on ne sauvegarde pas son chemin
        if file_label == 'a':
            pass
        else:
            if j < int((30000*20)/100) and i == test_indices[j]:
                file_names[dirname + '/' + filename] = "test"
                j += 1
            else:
                file_names[dirname + '/' + filename] = "train"
            i += 1


        TRAÎTEMENT DES DONNÉES

Une fois la récupération et le trie fait, nous allons traîter ces données. Il faut savoir qu'un fichier audio apporte beaucoup moins d'informations dans le domaine temporelle que le domaine fréquentielle. C'est pour cela que nous allons transformer les données brutes en mel-spectrogrammes afin de récupérer le maximum d'information utile pour le futur modèle. 

Ces mel-spectrogrammes vont être stockés dans 2 tableaux différents afin de différenciés les données pour l'entraînement et les données pour le test. 

Et enfin, nous créons en parallèles 2 autres tableaux, un pour les données d'entraînement et un pour les données de test, qui vont stockés la valeur des chiffres prononcés dans l'audio.

In [ ]:
# Attention ! L'exécution de cette partie du code peut être particulièrement longue ! (15-20 min)
labels = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
img_size = 64
cpt = 0
feature_train = []
label_train = []
feature_test = []
label_test = []

for cle, valeur in file_names.items():

    # Récupération de la valeur du chiffre prononcé qui est stockés dans le nom du fichier
    label = int(cle[21])
    
    # Chargement du fichier audio
    x , sr = librosa.load(cle)

    # STFT sur le fichier audio afin de créer un spectrogramme
    spectrogram = librosa.stft(x)

    # Création du mel-spectrogram
    sgram_mag, _ = librosa.magphase(spectrogram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sr)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)

    # Redimensionnement du mel-spectrogram afin que les données soit stockés dans une matrice carré
    img_arr = mel_sgram[...,::-1]
    resized_arr = cv2.resize(img_arr, (img_size, img_size))
    
    if (valeur == "test"):
        feature_test.append(resized_arr)
        label_test.append(label)
    else :
        feature_train.append(resized_arr)
        label_train.append(label)
    
    cpt += 1

    clear_output(wait=True)
    prog = round(cpt * 100 / 30000, 2)

    print("Progression : ", round(cpt * 100 / 30000, 2), "%")

        AFFICHAGE DES DONNÉES TRAÎTÉES

Grâce à matplotlib, nous allons afficher les mel-spectrograms afin de vérifier si aucune erreur n'a eu lieu lors du traitement et pour aussi comprendre les données que nous venons de créer.

<div style="text-align: center; margin-top: 50px"> <img src="./images/i-know.gif" width="500"> </div>

In [ ]:
plt.figure(figsize = (5,5))
plt.axis("off")
plt.imshow(feature_train[0])
plt.title(labels[label_train[0]])

plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.axis("off")
plt.imshow(feature_train[-1])
plt.title(labels[label_train[-1]])

plt.show()

        SAUVEGARDE DES DONNÉES

Maintenant que le traitement a été fait, il ne nous reste plus qu'à sauvegarder le travail fait. Pour sauvegarder les données, nous allons créer un fichier "data" qui va contenir un fichier "train" et "test" afin de séparer les données pour plus tard. Ensuite, dans chacun de ces fichiers, nous créons 10 nouveaux fichiers numérotés de 0 à 9 afin de séparer les données en fonction du chiffre enregistré dans l'audio. Et enfin, dans les fichiers numérotés, on enregistre les différents mel-spectrogrammes en format CSV qui est un format facile à mettre en place et suffisant pour les données que nous avons.

<div style="text-align: center; margin-top: 50px"> <img src="./images/meme-power-rangers.gif" width="500"> </div>

In [ ]:
# Création des dossiers data, train et test
parent_dir = './data'
os.mkdir(parent_dir)

train_dir = parent_dir + '/train'
test_dir = parent_dir + '/test'
os.mkdir(train_dir)
os.mkdir(test_dir)

# Création des dossiers de 0 à 9 dans les dossiers train et test
for i in range(len(labels)):
  file_dir = train_dir + '/' + labels[i]
  os.mkdir(file_dir)
  file_dir = test_dir + '/' + labels[i]
  os.mkdir(file_dir)

In [ ]:
# Sauvegarde des mel-spectrogram au format CSV dans le dossier train
for i in range(len(feature_train)):
  data = feature_train[i]
  path = train_dir + '/' + str(label_train[i]) + '/' + str(i) + '.csv'
  savetxt(path, data, delimiter=',')
  clear_output(wait=True)
  print("Progression de la sauvegarde des données d'entraînement : ", round((i+1) * 100 / len(feature_train), 2), "%")

# Sauvegarde des mel-spectrogram au format CSV dans le dossier test
for i in range(len(feature_test)):
  data = feature_test[i]
  path = test_dir + '/' + str(label_test[i]) + '/' + str(i) + '.csv'
  savetxt(path, data, delimiter=',')
  clear_output(wait=True)
  print("Progression de la sauvegarde des données d'entraînement : 100 %")
  print("Progression de la sauvegarde des données de test : ", round((i+1) * 100 / len(feature_test), 2), "%")

print("Données sauvegardées")